In [7]:
#!pip install fsspec s3fs pandas pyarrow pydantic
#!pip install ../libs/rag_eval_metrics/dist/rag_eval_metrics-0.1.0-py3-none-any.whl

In [9]:
from pydantic import TypeAdapter
import pandas as pd
from rag_eval_metrics.evaluate import evaluate

In [2]:
ground_truth = "s3://deltix-staging-dial-rag-eval/ground_truth_2.parquet"
answers = "s3://deltix-staging-dial-rag-eval/answers_2.parquet"
metrics = "s3://deltix-staging-dial-rag-eval/metrics_tmp.parquet"

In [3]:
pd.read_parquet(ground_truth).head()

,question,ground_truth_answer,facts,documents
0,How many staff members/resources do you have a...,The 10-K form does not provide specific inform...,"[As of December 31, 2022, we had approximately...",https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
1,2022 Annual Revenue (US $),The annual revenue for EPAM in 2022 was $4.825...,"[•We recorded revenues of $4.825 billion, or a...",https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
2,"Indicate your company's revenue, EBITDA, and n...",Based on the information provided in the 10-K ...,[The following table presents a summary of our...,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
3,Indicate your company's average turnover rate ...,The voluntary attrition rate for EPAM was 13.8...,[We focus on retaining and engaging top talent...,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
4,Who are your company's top five (5) customers?...,The specific names of EPAM's top five customer...,[The following table shows revenues from the t...,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...


In [4]:
pd.read_parquet(answers).head()

,question,answer,context,documents
0,How many staff members/resources do you have a...,The documents provided do not specify the exac...,[approximately 29% of the independent director...,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
1,2022 Annual Revenue (US $),"The annual revenue for the year 2022 was $4,82...",[The following table sets forth a summary of o...,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
2,"Indicate your company's revenue, EBITDA, and n...",The company's revenues for the past three fisc...,[The following table sets forth a summary of o...,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
3,Indicate your company's average turnover rate ...,"I'm sorry, but the documents provided do not c...","[As of December 31, 2022, $23.3 million of tot...",https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...
4,Who are your company's top five (5) customers?...,The documents provided do not specify the name...,"[customers, with 54.7% and 26.4% of our revenu...",https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...


In [5]:
metrics_dataset = evaluate(ground_truth, answers, metrics)
TypeAdapter(metrics_dataset.metadata).dump_json(indent=2)

DatasetMetadata(created_at=datetime.datetime(2024, 5, 16, 17, 20, 30, 285124), sources=[Dataset(metadata_path='s3://deltix-staging-dial-rag-eval/ground_truth_2.parquet.metadata.json', data_path='s3://deltix-staging-dial-rag-eval/ground_truth_2.parquet', metadata=DatasetMetadata(created_at=None, sources=[], tools={}, file_info={'ETag': '"c186de5a49e5b8b582efc321e38d9f27"', 'LastModified': datetime.datetime(2024, 5, 13, 18, 11, 45, tzinfo=tzutc()), 'size': 12026, 'name': 'deltix-staging-dial-rag-eval/ground_truth_2.parquet', 'type': 'file', 'StorageClass': 'STANDARD', 'VersionId': None, 'ContentType': 'binary/octet-stream'})), Dataset(metadata_path='s3://deltix-staging-dial-rag-eval/answers_2.parquet.metadata.json', data_path='s3://deltix-staging-dial-rag-eval/answers_2.parquet', metadata=DatasetMetadata(created_at=None, sources=[], tools={}, file_info={'ETag': '"e17bcf828808ce2b50b5cdc9b1285588"', 'LastModified': datetime.datetime(2024, 5, 13, 18, 10, 47, tzinfo=tzutc()), 'size': 31827,

In [6]:
metrics_df = metrics_dataset.read_dataframe()
metrics_df.head()

,documents,question,facts,context,facts_ranks,context_relevance,recall,precision,mrr,f1
0,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...,How many staff members/resources do you have a...,"[As of December 31, 2022, we had approximately...",[approximately 29% of the independent director...,"[-1, 3]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.5,0.071429,0.125,0.125000
1,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...,2022 Annual Revenue (US $),"[•We recorded revenues of $4.825 billion, or a...",[The following table sets forth a summary of o...,[7],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",1.0,0.076923,0.125,0.142857
2,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...,"Indicate your company's revenue, EBITDA, and n...",[The following table presents a summary of our...,[The following table sets forth a summary of o...,"[-1, -1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.000000,0.000,0.000000
3,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...,Indicate your company's average turnover rate ...,[We focus on retaining and engaging top talent...,"[As of December 31, 2022, $23.3 million of tot...",[-1],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.000000,0.000,0.000000
4,https://d18rn0p25nwr6d.cloudfront.net/CIK-0001...,Who are your company's top five (5) customers?...,[The following table shows revenues from the t...,"[customers, with 54.7% and 26.4% of our revenu...",[-1],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.000000,0.000,0.000000
